## Load Data

In [ ]:
import pandas as pd 
import numpy as  np 
import pyreadr
import os 
import openai

current_wd=os.getcwd()

In [ ]:
os.path.dirname(current_wd)

In [3]:
# Load RDS data 
result = pyreadr.read_r(f'{current_wd}/data/AffectivePandemic.Rds') 

df = result[None] # extract the pandas data frame 
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration__in_seconds_,Finished,RecordedDate,ResponseId,RecipientLastName,...,p_LimitedGov,p_LimitedGov1,p_ScienceFear,p_ScienceTooFast,p_Libertarianism,p_SelfReliance,p_StrangerCutLine,p_PutOnShow,h_communityMask,h_communityGroceryMask
0,2021-11-19 13:48:20,2021-11-19 13:53:41,0.0,73.216.114.158,100.0,320.0,1.0,2021-11-19 13:53:41,R_1espJB9ZHIg9ngo,,...,0.0,0.0,-5.0,0.0,-4.0,0.0,0.0,0.0,9.0,0.0
1,2021-11-19 13:47:04,2021-11-19 13:54:37,0.0,108.185.238.236,100.0,452.0,1.0,2021-11-19 13:54:37,R_1Fyz86q35YBQDUI,,...,-1.0,0.0,-5.0,0.0,-5.0,0.0,0.0,0.0,6.0,1.0
2,2021-11-19 13:45:52,2021-11-19 13:55:02,0.0,72.186.41.34,100.0,549.0,1.0,2021-11-19 13:55:02,R_XoWyI6XGT5hdf5T,,...,0.0,1.0,-2.0,0.0,-5.0,0.0,0.0,0.0,9.0,0.0
3,2021-11-19 13:46:25,2021-11-19 13:55:56,0.0,205.154.88.148,100.0,570.0,1.0,2021-11-19 13:55:56,R_87YNShWdZrSGlzP,,...,0.0,0.0,-4.0,0.0,-5.0,0.0,0.0,0.0,6.0,1.0
4,2021-11-19 13:46:08,2021-11-19 13:55:58,0.0,172.58.144.10,100.0,589.0,1.0,2021-11-19 13:55:58,R_29mggzhfSdfqZT6,,...,-1.0,0.0,-2.0,0.0,-3.0,1.0,1.0,0.0,NaN,0.0


In [ ]:
len(list((df.columns)))

print(list((df.columns)))

## Data Preprocessing

In [ ]:
# Check for missingness 
missingness_pct = df.isna().sum()/len(df)*100
missingness_pct
missingness_pct[missingness_pct!=0].plot(kind='bar')

In [ ]:
missingness_pct[missingness_pct!=0]

## Setup OpenAI


In [19]:
from openai import OpenAI
import openai

client = OpenAI()
openai.api_key = '

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
) 

ImportError: cannot import name 'OpenAI' from 'openai' (/Users/tranghoang/opt/anaconda3/lib/python3.9/site-packages/openai/__init__.py)

In [ ]:
response

In [ ]:
response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Write a tagline for an ice cream shop."
)

In [ ]:
response

## Grab Columns for analysis
- Find basic information columns like Age, Education etc


- Columns' definition: 
    - QRace1 -  White
    - QRace2 -  Black or African-American
    - QRace3 - Asian or Asian-American
    - QRace4 - American Indian or Alaska Native
    - QRace5 - Native Hawaiian or Pacific Islander
    - QRace6 -  Middle Eastern or North African
    - QRace7 - Other
    

In [ ]:
[col for col in df.columns if 'vax' in col.lower()]

In [6]:
df.apply(get_race_label, axis=1)

0       White
1       White
2       White
3       White
4       White
        ...  
3030    White
3031    White
3032    White
3033    White
3034    White
Length: 3035, dtype: object

In [7]:
demo_col= ['d_age', 'Qgender','d_Educat', 'QPID', 'h_Vaxed']

gender = {1.0:'Male', 2.0 : 'Female', 3.0:'Other'}

race_labels= {
    "Qrace_1" : "White"
    , "Qrace_2": "Black or African-American"
    , "Qrace_3" : "Asian or Asian-American"
    , "Qrace_4" : "American Indian or Alaska Native"
    , "Qrace_5" : "Native Hawaiian or Pacific Islander"
    , "Qrace_6" : "Middle Eastern or North African"
    , "Qrace_7": "Other"
}
def get_race_label(row): 
    for key, value in race_labels.items(): 
        print(key, value)
        if row[key]==1:
            return value
        else:
            return "Unknow"

vax_label = {1.0:"Vaxed", 0.0:"No Vaxed"}
        
# Classify race gender and vaccination label
df['race'] = df.apply(get_race_label, axis=1)
df['gender']=df['Qgender'].map(gender) 
df["vaccination"] = df['h_Vaxed'].map(vax_label)


demo_col.append('gender')
demo_col.append('race')
demo_col.append("vaccination")
df_gpt= df[demo_col]

Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_1 White
Qrace_

In [5]:
df_gpt.head()


,d_age,Qgender,d_Educat,QPID,h_Vaxed,gender,race,vaccination
0,28.0,1.0,College,Independent,1.0,Male,White,Vaxed
1,42.0,2.0,College,Democrat,1.0,Female,White,Vaxed
2,23.0,2.0,College,Independent,0.0,Female,White,No Vaxed
3,68.0,1.0,Graduate Degree,Democrat,1.0,Male,White,Vaxed
4,23.0,2.0,HS or Less,Independent,1.0,Female,White,Vaxed


### Potential Cleaning and data processing before training 
- make sure that the `d_Race_White` matches with the `Qrace_1` and `d_Race_Black` and `Qrace2`
- 

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)
type(sports_dataset)

In [ ]:
# Request Prompt 
def request_completion(prompt):
    
    completion_response =   openai.Completion.create(
                            prompt=prompt,
                            temperature=0,
                            max_tokens=5,
                            top_p=1,
                            frequency_penalty=0,
                            presence_penalty=0,
                            model=COMPLETIONS_MODEL
                            )
        
    return completion_response


In [ ]:
df_gpt['race'].value_counts()

In [ ]:

df_gpt.tail()

In [14]:
import openai

test_df = df_gpt.iloc[0]
openai.api_key = ''

#Write prompt
prompt = "Based on the provided information below about a person, do you think they are vaccinated for COVID-19"
#prompt= "you help me classify if this person would be vaccinated for COVID-19 \n"

race = test_df['race']
gender= test_df['gender']
age = test_df['d_age']
education = test_df['d_Educat']
prompt += f'A "{race}" "{gender}" at the age of "{age}" with "{education}" degree'
print(prompt)
response= openai.OpenAI.chat.Completion.create(
    engine = "gpt-3.5-turbo", 
    prompt=prompt
    , max_tokens=100 
    , temperature=0.5)
print(response.choices[0].text.strip())

Based on the provided information below about a person, do you think they are vaccinated for COVID-19A "White" "Male" at the age of "28.0" with "College" degree


AttributeError: module 'openai' has no attribute 'chat'

In [ ]:
promt= "The following is a list of demographic with their corresponding COVID 19 vaccinations categories: \n\n "
examples = df_gpt.iloc[0:10]

for i in range(examples.shape[0]): 
    race = examples.loc[i, 'race']
    gender= examples.loc[i, 'gender']
    age = examples.loc[i,'d_age']
    education = examples.loc[i,'d_Educat']
    vaccination = examples.loc[i,'vaccination']
    prompt += f'{i+1}) A "{race}" "{gender}" at the age of "{age}" with "{education}" degree: {vaccination} \n '
print(prompt)

In [ ]:
examples.loc[1,'gender']

In [ ]:
import openai
test_df = df_gpt.iloc[0:10]

def classify_object(examples, object_to_classify):
    # Construct the prompt with examples
    
    prompt= "The following is a list of demographic with their corresponding COVID 19 vaccinations categories: \n\n "
    
    for i in range(examples.shape[0]): 
        race = examples.loc[i, 'race']
        gender= examples.loc[i, 'gender']
        age = examples.loc[i,'d_age']
        education = examples.loc[i,'d_Educat']
        vaccination = examples.loc[i,'vaccination']
        prompt += f'{i+1}) A "{race}" "{gender}" at the age of "{age}" with "{education}" degree: {vaccination} \n '
    
    
    prompt += f'\n"{object_to_classify}": '
    print(prompt)
    response = openai.Completion.create(
        engine="text-davinci-003",  # Replace with the model of your choice
        prompt=prompt,
        max_tokens=60,  # You may need to adjust this
        temperature=0,  # A lower temperature can make the response more deterministic
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # Return the classification result
    return response.choices[0].text.strip()


# Object to classify
objects= df_gpt.iloc[11]
race = test_df['race']
gender= test_df['gender']
age = test_df['d_age']
education = test_df['d_Educat']

object_to_classify = f'A "{race}" "{gender}" at the age of "{age}" with "{education}" degree: '
# Get the classification
classification = classify_object(df_gpt[0:10], object_to_classify)
print(f'The object "{object_to_classify}" is classified as: "{classification}"')